# Habit Tracker App using ibm-granite/granite-20b-functioncalling

This notebook is heavily based on this one: [https://github.com/curiousily/AI-Bootcamp/blob/master/16.llm-function-calling.ipynb](16.llm-function-calling.ipynb). Also do checkout the amazing [YouTube tutorial on tool use](https://www.youtube.com/watch?v=5XFqrAG0OIk) by its author. 

What is different in this version is basically the model and the backend using vLLM. The model by default comes with a `chat_template` that does work for the example code on the Huggingface page, but doesn't support several turns. Also the tool use API in vLLM doesn't seem to be 100% compatible with the `openai` client libraries yet. Because of these differences, this notebook used the completions API instead of the chat API and parses the answers manually


In [70]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.44.0 openai==1.40.3 --progress-bar off

In [1]:
import json
import os
import sqlite3
from dataclasses import dataclass
from datetime import date
from enum import Enum, auto
from typing import List, Set

In [2]:
import openai
openai.api_key = '...'
openai.base_url = "http://0.0.0.0:8000/v1/"
MODEL = "ibm-granite/granite-20b-functioncalling"

In [3]:
DB_NAME = "habit_tracker.db"


class DayOfWeek(Enum):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()


@dataclass
class Habit:
    id: int
    name: str
    repeat_frequency: Set[DayOfWeek]
    tags: List[str]


@dataclass
class DailyHabitEntry:
    id: int
    name: str
    tags: List[str]
    is_completed: bool


def get_connection():
    return sqlite3.connect(DB_NAME)


def create_tables():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS habits (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                repeat_frequency TEXT NOT NULL,
                tags TEXT NOT NULL
            )
        """
        )
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS completions (
                habit_id INTEGER,
                completion_date TEXT,
                PRIMARY KEY (habit_id, completion_date),
                FOREIGN KEY (habit_id) REFERENCES habits (id)
            )
        """
        )
        conn.commit()


def list_habits() -> List[Habit]:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM habits")
        return [
            Habit(
                id,
                name,
                {DayOfWeek[day] for day in freq.split(",")},
                tags.split(","),
            )
            for id, name, freq, tags in cursor.fetchall()
        ]


def habits_for_date(date: date) -> List[DailyHabitEntry]:
    weekday = DayOfWeek(date.weekday() + 1).name
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT h.id, h.name, h.tags, c.completion_date IS NOT NULL as completed
            FROM habits h
            LEFT JOIN completions c ON h.id = c.habit_id AND c.completion_date = ?
            WHERE instr(h.repeat_frequency, ?) > 0
        """,
            (date.isoformat(), weekday),
        )
        return [
            DailyHabitEntry(id, name, tags.split(","), bool(completed))
            for id, name, tags, completed in cursor.fetchall()
        ]


def complete_habit(habit_id: int, completion_date: date):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT OR REPLACE INTO completions (habit_id, completion_date)
            VALUES (?, ?)
        """,
            (habit_id, completion_date.isoformat()),
        )
        conn.commit()


def add_habit(name: str, repeat_frequency: Set[DayOfWeek], tags: List[str] = []) -> int:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO habits (name, repeat_frequency, tags)
            VALUES (?, ?, ?)
        """,
            (name, ",".join(day.name for day in repeat_frequency), ",".join(tags)),
        )
        conn.commit()
        return cursor.lastrowid


def show_habits_for_date(date: date):
    print(f"Habits for {date}:")
    for entry in habits_for_date(date):
        status = "Completed" if entry.is_completed else "Not completed"
        print(f"- {entry.name} (ID: {entry.id}): {status}")
        print(f"  Tags: {', '.join(entry.tags)}")


create_tables()

In [4]:
add_habit(
    "Hit the gym",
    {DayOfWeek.MONDAY, DayOfWeek.WEDNESDAY, DayOfWeek.FRIDAY},
    ["exercise", "fitness"],
)
add_habit("Feed the llamas", {DayOfWeek.SATURDAY, DayOfWeek.SUNDAY}, ["diet"])

2

In [5]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>, <DayOfWeek.MONDAY: 1>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SUNDAY: 7>, <DayOfWeek.SATURDAY: 6>}, tags=['diet'])]

In [6]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "add_habit",
            "description": "Add a new habit. Returns the id for the habit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the habit",
                    },
                    "repeat_frequency": {
                        "type": "array",
                        "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']",
                        "items": {
                            "type": "string",
                            "enum": [
                                "MONDAY",
                                "TUESDAY",
                                "WEDNESDAY",
                                "THURSDAY",
                                "FRIDAY",
                                "SATURDAY",
                                "SUNDAY",
                            ],
                        },
                    },
                    "tags": {
                        "type": "array",
                        "description": "List of tags, e.g. ['health', 'fitness']",
                    },
                },
                "required": ["name", "repeat_frequency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_habits",
            "description": "Returns a list of all available habits",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "habits_for_date",
            "description": "Returns a list of habits scheduled for a date",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "str",
                        "description": "Date for which to display scheduled habits in ISO format e.g. 2024-11-23",
                    }
                },
                "required": ["date"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "complete_habit",
            "description": "Completes a specific habit for a given date",
            "parameters": {
                "type": "object",
                "properties": {
                    "habit_id": {
                        "type": "integer",
                        "description": "Id of the habit, e.g. 1",
                    },
                    "completion_date": {
                        "type": "str",
                        "description": "Date for completion of the habit in ISO format e.g. 2024-11-23",
                    },
                },
                "required": ["habit_id", "completion_date"],
            },
        },
    },
]

In [7]:
AVAILABLE_FUNCTIONS = {
    "add_habit": add_habit,
    "list_habits": list_habits,
    "habits_for_date": habits_for_date,
    "complete_habit": complete_habit,
}

In [8]:
argument_mapping = {}
argument_mapping["repeat_frequency"] = lambda day_names: [
    DayOfWeek[d] for d in day_names
]
argument_mapping["date"] = lambda d: date.fromisoformat(d)
argument_mapping["completion_date"] = lambda d: date.fromisoformat(d)

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "ibm-granite/granite-20b-functioncalling"
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/maxdebayser/.pyenv/versions/3.11.6/envs/tool_use/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [48]:
func_list = [f["function"] for f in TOOLS]
user_prompt = "Add a new habit for Reading a book every weekday #learning"

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant with access to the following functions. Use them if required -"
    },
    {
        "role": "user",
        "content": user_prompt,
    },
#    Uncomment to see the mechanics of the function call in action    
#    {
#        "role": "assistant",
#        "content": """<function_call> {"name": "add_habit", "arguments": {"name": "Reading a book", "repeat_frequency": ["MONDAY", "WEDNESDAY", "FRIDAY"], "tags": ["learning"]}}""",
#    },
#    {
#        "role": "tool",
#        "content": "<function_response> 3",
#    },
]

tokenizer.chat_template = """{% set funcstr = tools|map("tojson")|join(\'\n\') %}
{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{- '\nUSER: ' + message['content'] }}
    {% elif message['role'] == 'system' %}
        {{- 'SYSTEM: ' + message['content'] + '\n<|function_call_library|>\n' + funcstr + '\n\nIf none of the functions are relevant or the given question lacks the parameters required by the function, please output "<function_call> {"name": "no_function", "arguments": {}}".\n' }}
    {% elif message['role'] == 'assistant' %}
        {{- '\nASSISTANT: ' + message['content'] + ' <|endoftext|>'  }}
    {% elif message['role'] == 'tool' %}
        {{- message['content'] }}
    {% endif %}
    {% if loop.last and add_generation_prompt %}
        {{- '\nASSISTANT: ' }}
    {% endif %}
{% endfor %}
"""
instruction = tokenizer.apply_chat_template(messages, tools=func_list, tokenize=False, add_generation_prompt=True)
print(instruction)

response = openai.completions.create(
    model=MODEL,
    prompt=instruction,
    temperature=0,
    max_tokens=4096,
)
response


SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
<|function_call_library|>
{"name": "add_habit", "description": "Add a new habit. Returns the id for the habit.", "parameters": {"type": "object", "properties": {"name": {"type": "string", "description": "Name of the habit"}, "repeat_frequency": {"type": "array", "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']", "items": {"type": "string", "enum": ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]}}, "tags": {"type": "array", "description": "List of tags, e.g. ['health', 'fitness']"}}, "required": ["name", "repeat_frequency"]}}
{"name": "list_habits", "description": "Returns a list of all available habits"}
{"name": "habits_for_date", "description": "Returns a list of habits scheduled for a date", "parameters": {"type": "object", "properties": {"date": {"type": "str", "description": "Date for which to display scheduled habits in IS

Completion(id='cmpl-b81b9c61c8904d9185512cddc58af8aa', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='<function_call> {"name": "add_habit", "arguments": {"name": "Reading a book", "repeat_frequency": ["MONDAY", "WEDNESDAY", "FRIDAY"], "tags": ["learning"]}} ', stop_reason=None)], created=1723423566, model='ibm-granite/granite-20b-functioncalling', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=54, prompt_tokens=506, total_tokens=560))

In [24]:
tool_calls = [json.loads(s) for s in response.choices[0].text.split("<function_call>") if s.strip()]

In [25]:
for tool_call in tool_calls:
    function_name = tool_call["name"]
    function_to_call = AVAILABLE_FUNCTIONS[function_name]
    function_args = tool_call["arguments"]

In [26]:
function_args

{'name': 'Reading a book',
 'repeat_frequency': ['MONDAY', 'WEDNESDAY', 'FRIDAY'],
 'tags': ['learning']}

In [27]:
arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
arguments_to_map

['repeat_frequency']

In [28]:
for arg_name in arguments_to_map:
    function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])

In [29]:
print(function_args)

{'name': 'Reading a book', 'repeat_frequency': [<DayOfWeek.MONDAY: 1>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>], 'tags': ['learning']}


In [30]:
function_response = function_to_call(**function_args)
function_response

3

In [31]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>, <DayOfWeek.MONDAY: 1>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SUNDAY: 7>, <DayOfWeek.SATURDAY: 6>}, tags=['diet']),
 Habit(id=3, name='Reading a book', repeat_frequency={<DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>, <DayOfWeek.MONDAY: 1>}, tags=['learning'])]

In [32]:
def map_arguments(function_args: dict, argument_mapping: dict = argument_mapping):
    arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
    for arg_name in arguments_to_map:
        function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])
    return function_args

In [63]:
from dataclasses import dataclass, asdict, is_dataclass

class DataClassEncoder(json.JSONEncoder):
    def default(self, obj):
        if is_dataclass(obj):
            return asdict(obj)
        return super().default(obj)

def call_model(messages):
    instruction = tokenizer.apply_chat_template(messages, tools=func_list, tokenize=False, add_generation_prompt=True)
    print(instruction)
    
    response = openai.completions.create(
        model=MODEL,
        prompt=instruction,
        temperature=0,
        max_tokens=4096,
    )

    assert len(response.choices) == 1, "Don't know how to handle multiple choices"
    return response.choices[0].text

def call_function(prompt, messages: List) -> List:
    messages.append(
        {
            "role": "user",
            "content": prompt,
        }
    )
    
    response_message = call_model(messages)
    messages.append({"role": "assistant", "content": response_message})

    tool_calls = [json.loads(s) for s in response_message.split("<function_call>") if s.strip()]

    for tool_call in tool_calls:
        function_name = tool_call["name"]
        function_to_call = AVAILABLE_FUNCTIONS[function_name]
        function_args = tool_call["arguments"]
        function_args = map_arguments(function_args, argument_mapping)

        function_response = function_to_call(**function_args)
        if function_response is not None:
            function_response = json.dumps(function_response, cls=DataClassEncoder)
        else:
            function_response = "{}"
        messages.append(
            {
                "role": "tool",
                "content": "<function_response> " + function_response,
            }
        )
        response_message = call_model(messages)
        messages.append({"role": "assistant", "content": response_message})
    return messages

def start_messages():
    return [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to the following functions. Use them if required -"
        }
    ]
messages = start_messages()

In [64]:
today = date(2024, 7, 26)
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Not completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning


In [65]:
user_prompt = f"Show all habits for today - {today.isoformat()}"
call_function(user_prompt, messages)

SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
<|function_call_library|>
{"name": "add_habit", "description": "Add a new habit. Returns the id for the habit.", "parameters": {"type": "object", "properties": {"name": {"type": "string", "description": "Name of the habit"}, "repeat_frequency": {"type": "array", "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']", "items": {"type": "string", "enum": ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]}}, "tags": {"type": "array", "description": "List of tags, e.g. ['health', 'fitness']"}}, "required": ["name", "repeat_frequency"]}}
{"name": "list_habits", "description": "Returns a list of all available habits"}
{"name": "habits_for_date", "description": "Returns a list of habits scheduled for a date", "parameters": {"type": "object", "properties": {"date": {"type": "str", "description": "Date for which to display scheduled habits in IS

[{'role': 'system',
  'content': 'You are a helpful assistant with access to the following functions. Use them if required -'},
 {'role': 'user', 'content': 'Show all habits for today - 2024-07-26'},
 {'role': 'assistant',
  'content': '<function_call> {"name": "habits_for_date", "arguments": {"date": "2024-07-26"}}'},
 {'role': 'tool',
  'content': '<function_response> [{"id": 1, "name": "Hit the gym", "tags": ["exercise", "fitness"], "is_completed": false}, {"id": 3, "name": "Reading a book", "tags": ["learning"], "is_completed": false}]'},
 {'role': 'assistant',
  'content': '\n<function_call> {"name": "no_function", "arguments": {}} '}]

In [67]:
user_prompt = f"Complete the gym habit for {today.isoformat()}"
call_function(user_prompt, start_messages())

SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
<|function_call_library|>
{"name": "add_habit", "description": "Add a new habit. Returns the id for the habit.", "parameters": {"type": "object", "properties": {"name": {"type": "string", "description": "Name of the habit"}, "repeat_frequency": {"type": "array", "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']", "items": {"type": "string", "enum": ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]}}, "tags": {"type": "array", "description": "List of tags, e.g. ['health', 'fitness']"}}, "required": ["name", "repeat_frequency"]}}
{"name": "list_habits", "description": "Returns a list of all available habits"}
{"name": "habits_for_date", "description": "Returns a list of habits scheduled for a date", "parameters": {"type": "object", "properties": {"date": {"type": "str", "description": "Date for which to display scheduled habits in IS

[{'role': 'system',
  'content': 'You are a helpful assistant with access to the following functions. Use them if required -'},
 {'role': 'user', 'content': 'Complete the gym habit for 2024-07-26'},
 {'role': 'assistant',
  'content': '<function_call> {"name": "complete_habit", "arguments": {"habit_id": 1, "completion_date": "2024-07-26"}}'},
 {'role': 'tool', 'content': '<function_response> {}'},
 {'role': 'assistant',
  'content': "\nThe habit 'gym' has been completed for 2024-07-26. "}]

In [69]:
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning
